# **Introduction**

***Howdy, Welcome to the Titanic***

**To whom does this notebook appeal to?**<br><br>
If you are just starting with WNNC's HELLO FOSS and want to do beginner task in ML, you can try this project!

Data reading and analyzation has been done and a neural network is implemented from scratch in Numpy to predict whether or not they survived the sinking of the Titanic. The backward propogation part of the network has not been completed and we need you to code the rest of the function to return the gradients. We have also included `gender_submission.csv`, a set of predictions that assume all and only female passengers survive, as an example of what a submission file should look like.

Head over to [this cell](#main) to implement back propogation of neural network and then add code for writing the output to file `predictions-ann.csv` [here](#main2). 

![](https://faithmag.com/sites/default/files/styles/article_full/public/2018-09/titanic2.jpg?h=6521bd5e&itok=H8td6QVv)

**Basic Imports**

In [58]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./titanic'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

./titanic/test.csv
./titanic/output.csv
./titanic/grader.py
./titanic/gender_submission.csv
./titanic/train.csv


In [59]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [60]:
df = pd.read_csv("./titanic/train.csv")
#print(df)

### Analyze Data

In [61]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


To make a quick neural network using the data above,<br>
we can easily create a neural network using the following the columns:<br>
'**Age**', '**Sex**', '**Fare**', '**Pclass**', '**SibSp**', '**Parch**'

In [62]:
# lets take out first the label
train_y = df['Survived']
train_y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [63]:
# function to filter the age, sex, fare pclass, sibsp, parch columns
def get_data(data):
    # take only this specific column
    data = data[['Age', 'Sex', 'Fare', 'Pclass', 'SibSp', 'Parch']]
    
    # replace male by 1, female by 0
    data.replace({ 'male' : 1, 'female' : 0 }, inplace=True)
    
    # replace null/nan data by the mean (age and fare columns)
    data['Fare'].fillna(int(data['Fare'].mean()), inplace=True)
    data['Age'].fillna(int(data['Age'].mean()), inplace=True)
    
    # transform into a numpy array
    data = data.to_numpy().astype(float)
    
    # normalize (make sure the data is between -1 and 1)
    for i in range(data.shape[1]):
        data[:,i] = (data[:,i] - data[:,i].mean()) / data[:,i].std()
    
    return data

In [64]:
train_x = get_data(df)
print(train_x)

[[-0.58165904  0.73769513 -0.50244517  0.82737724  0.43279337 -0.47367361]
 [ 0.64932701 -1.35557354  0.78684529 -1.56610693  0.43279337 -0.47367361]
 [-0.27391253 -1.35557354 -0.48885426  0.82737724 -0.4745452  -0.47367361]
 ...
 [-0.04310264 -1.35557354 -0.17626324  0.82737724  0.43279337  2.00893337]
 [-0.27391253  0.73769513 -0.04438104 -1.56610693 -0.4745452  -0.47367361]
 [ 0.18770724  0.73769513 -0.49237783  0.82737724 -0.4745452  -0.47367361]]


In [65]:
print(train_x.shape)

(891, 6)


Shape will show us the number of rows and columns (891 and 6)

In [66]:
# same for the labels (contains 0 - 1 if the victim survived or not)
train_y = train_y.to_numpy()
train_y = train_y.reshape(train_x.shape[0],1)
print(train_y.shape)

(891, 1)


### Neural Network

In [67]:
# the activation function and derivative of the action function
def sigmoid(x):
    return 1/(1+np.exp(-x))

def dsigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [68]:
# the loss function and its derivative
def loss_fn(y, y_hat):
    return 1/2 * (y - y_hat) ** 2

def dloss_fn(y, y_hat):
    return (y - y_hat)

In [69]:
# number of rows
instances = train_x.shape[0]

# number oof columns
attributes = train_x.shape[1]

# number of hidden node for first layer 
hidden_nodes = 8

# number of hidden node for second layer
hidden_nodes_two = 4

# number of output labels 
output_labels = 1

### Inititate the weights/biases

In [70]:
w1 = np.random.rand(attributes,hidden_nodes)
b1 = np.random.randn(hidden_nodes)

w2 = np.random.rand(hidden_nodes,hidden_nodes_two)
b2 = np.random.randn(hidden_nodes_two)

w3 = np.random.rand(hidden_nodes_two, output_labels)
b3 = np.random.randn(output_labels)

theta = w1, w2, w3, b1, b2, b3

### Neural Network Forward Propogation

In [71]:
def forward(x, theta):
    w1, w2, w3, b1, b2, b3 = theta
    
    k = np.dot(x, w1) + b1
    l = sigmoid(k)
    
    m = np.dot(l, w2) + b2
    n = sigmoid(m)
    
    o = np.dot(n, w3) + b3
    p = sigmoid(o)
    
    return k, l, m, n, o, p

### Neural Network Backward Propogation
<a id='main'></a>

In [72]:
def backward(x, y, sigma, theta):
    w1, w2, w3, b1, b2, b3 = theta
    z1, a1, z2, a2, z3, a3 = sigma
    dz3 = dloss_fn(a3, y)
    dw3 = a2.T@dz3
    db3 = dz3.copy()
    dz2 = dz3@w3.T * dsigmoid(z2)
    dw2 = a1.T@dz2
    db2 = dz2.copy()
    dz1 = dz2@w2.T * dsigmoid(z1)
    dw1 = x.T@dz1
    db1 = dz1.copy()
    #complete this function
     
    return dw1, dw2, dw3, db1, db2, db3

#### Use the avg of the gradients for the derivative of each bias

In [73]:
def avg_bias(grads):
    dw1, dw2, dw3, db1, db2, db3 = grads
    db1 = db1.mean(axis=0)
    db2 = db2.mean(axis=0)
    db3 = db3.mean(axis=0)
    return dw1, dw2, dw3, db1, db2, db3

#### Use the SGD in order to optimize the weights and biases

In [74]:
def optimize(theta, grads, lr=0.001):
    dw1, dw2, dw3, db1, db2, db3 = grads
    w1, w2, w3, b1, b2, b3 = theta
    
    w1 -= dw1 * lr
    w2 -= dw2 * lr
    w3 -= dw3 * lr
    b1 -= db1 * lr
    b2 -= db2 * lr
    b3 -= db3 * lr
    
    return w1, w2, w3, b1, b2, b3

#### Make Prediction

In [75]:
# return 1 if the prediction is higher than 0.5
# return 0 if not
def predict(x, theta):
    predict = forward(x, theta)[-1]
    return np.where(predict > 0.5, 1, 0)

### Training the Model

In [76]:
# time to train our model
for epoch in range(1000):
    
    sigma = forward(train_x, theta)
    grads = backward(train_x, train_y, sigma, theta)
    theta = optimize(theta, avg_bias(grads))
    
    if(epoch % 100 == 0):
        print(loss_fn(sigma[-1], train_y).mean())

0.2498637203729087
0.08513839237507682
0.07222216206544173
0.06967866959316654
0.06882210601447634
0.06814783557337917
0.06750558396086936
0.06688715675334578
0.06631522225506462
0.0658241983650883


#### Print the accuracy

In [77]:
pred_train = predict(train_x, theta)
num = pred_train.shape[0]
acc=0
for i in range(num):
     acc+=(pred_train[i,0]==train_y[i,0])
print("Accuracy on training set: ", 100*acc/num)

Accuracy on training set:  82.26711560044893


### Time to train the test data

In [78]:
test_df = pd.read_csv(dirname + "/test.csv")
test_x = get_data(test_df)

In [79]:
# Get test data predictions
test_preds = predict(test_x, theta)

In [80]:
# Add passengers ids to the test predictions
passenger_ids = test_df['PassengerId'].to_numpy()


#### Output the prediction to submission file
<a id='main2'></a>

In [81]:
# combine passenger ids with the predictions
combined = np.concatenate((passenger_ids.reshape(-1,1) , test_preds), axis = 1)
# convert array to dataframe
result_df = pd.DataFrame(combined, columns=["Pass_id","Survived"])
# save the result to predictions-ann.csv
result_df.to_csv("predictions-ann.csv")
print(result_df)

     Pass_id  Survived
0        892         0
1        893         0
2        894         0
3        895         0
4        896         1
..       ...       ...
413     1305         0
414     1306         1
415     1307         0
416     1308         0
417     1309         0

[418 rows x 2 columns]


In [82]:
def grade():
    data_to_be_graded = pd.read_csv('/content/predictions-ann.csv')
    expected = pd.read_csv('/content/titanic/output.csv')
    accuracy = sum(expected['Survived'].values == data_to_be_graded['Survived'].values) / len(expected)
    print("Your accuracy is {}%".format(round(accuracy * 100, 2)))

In [83]:
grade()

Your accuracy is 78.95%
